In [16]:
! pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification')

Skipping, found downloaded files in "./covid-19-nlp-text-classification" (use force=True to force download)


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import string
from tensorflow.keras import layers
from tensorflow.keras import losses
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [19]:
df = pd.read_csv('covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='latin-1')

In [20]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [21]:
def clean_data(data):
    stop_wd = stopwords.words('english')
    wl = WordNetLemmatizer()
    vocab_size = 5000
    res = []
    tweets = []
    for i in data['OriginalTweet']:
        tweets.append(i)
    # print('tweets ',tweets)
    for text in tweets:
        text = re.sub(r'https?://\S+', ' ', text)
        text = re.sub(r'@\w*', ' ' , text)
        text = re.sub(r'#\w*', ' ' , text)
        text = re.sub(r"'s\b", "", text)
        text = re.sub(r"[^a-zA-Z0-9\s]", ' ', text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        temp = []
        for word in text:
            if word not in stop_wd:
                temp.append(word)
        text = temp
        temp = []
        for word in text:
            temp.append(wl.lemmatize(word))
        text = temp
        temp=[]
        for word in text:
            temp.append(one_hot(word,vocab_size)[0])
        text = temp
        text = pad_sequences([text],padding='pre',maxlen=50)
        # print(text)
        res.append(text[0])
    # res = np.array(res[0])
    return np.array(res)

In [22]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
x = clean_data(df)

In [24]:
x.shape

(41157, 50)

In [25]:
def clean(obj):
    if obj=='Positive' or obj=='Extremely Positive':
        return 0
    elif obj=='Negative' or obj=='Extremely Negative':
        return 1
    elif obj =='Neutral':
        return 2

df['Sentiment'] = df['Sentiment'].apply(clean)

In [26]:
y = df['Sentiment']

In [37]:
vocab_size=5000
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,40,input_length=50))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dense(3,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 3)                 303       
                                                                 
Total params: 256,703
Trainable params: 256,703
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
x.shape , y.shape

((41157, 50), (41157,))

In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [42]:
with tf.device('GPU:0/'):
  model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
1029/1029 [==============================] - 9s 8ms/step - loss: 0.3239 - accuracy: 0.8996 - val_loss: 0.3351 - val_accuracy: 0.8925
Epoch 2/10
1029/1029 [==============================] - 8s 8ms/step - loss: 0.2301 - accuracy: 0.9280 - val_loss: 0.3991 - val_accuracy: 0.8712
Epoch 3/10
1029/1029 [==============================] - 8s 8ms/step - loss: 0.1783 - accuracy: 0.9436 - val_loss: 0.4670 - val_accuracy: 0.8547
Epoch 4/10
1029/1029 [==============================] - 8s 8ms/step - loss: 0.1363 - accuracy: 0.9576 - val_loss: 0.5472 - val_accuracy: 0.8423
Epoch 5/10
1029/1029 [==============================] - 8s 7ms/step - loss: 0.1019 - accuracy: 0.9695 - val_loss: 0.6512 - val_accuracy: 0.8294
Epoch 6/10
1029/1029 [==============================] - 7s 7ms/step - loss: 0.0813 - accuracy: 0.9747 - val_loss: 0.7166 - val_accuracy: 0.8192
Epoch 7/10
1029/1029 [==============================] - 9s 8ms/step - loss: 0.0671 - accuracy: 0.9790 - val_loss: 0.8056 - val_accuracy:

In [43]:
y_pred = model.predict(x_test)

258/258 [==============================] - 1s 4ms/step


In [44]:
y_pred  = y_pred.argmax( axis=-1).astype('int')

In [45]:
y_pred

array([2, 1, 0, ..., 2, 2, 0])

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2878,  558,  190],
       [ 278, 2607,  195],
       [ 184,  259, 1083]])

In [47]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_pred,y_test)
print('Accuracy: %.3f' % (acc*100))

Accuracy: 79.786


In [48]:
text="Covid is very bad #GoCORONA"

d = {'OriginalTweet':[text]}
test_data = pd.DataFrame.from_dict(d)
with tf.device('CPU:0/'):
    val = model.predict(np.array(clean_data(test_data)))
    ans = val.argmax(axis=-1)
    if ans==0:
        print('Positive')
    elif ans==1:
        print('Negative')
    else:
        print('Neutral')

1/1 [==============================] - 0s 123ms/step
Negative
